In [1]:
!pip install ultralytics -q

# Import Modules

In [ ]:
import pandas as pd
import numpy as np
import os
import subprocess
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Affichage d'images et de vidéos 
import IPython
from IPython.display import Video, display
%matplotlib inline 

In [ ]:
# Chemin vers la vidéo locale à afficher
video_path = '/Users/macbook/Desktop/Project1/vehicle-counting.mp4'
# Affichage de la vidéo avec une largeur et une hauteur spécifiées
display(Video(video_path, embed=True, width=640, height=360))

In [ ]:
# Chargement du modèle YOLO
model = YOLO('yolov8x.pt')

# Obtention des noms de classes à partir du modèle
dict_classes = model.model.names

In [ ]:
# Fonction pour redimensionner une image 
def resize_frame(frame, scale_percent):
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
    return resized

# Detecting vehicles

In [ ]:
#configuration
verbose = False
scale_percent = 50

# Lecture de la vidéo avec OpenCV (cv2)
video = cv2.VideoCapture(video_path)

# Objets à détecter avec YOLO
class_IDS = [2, 3, 5, 7] 
centers_old = {}
centers_new = {}
obj_id = 0 
in_objects_counter = dict.fromkeys(class_IDS, 0)   # Compteur d'objets entrant par classe
out_objects_counter = dict.fromkeys(class_IDS, 0)  # Compteur d'objets sortant par classe
end = []
frames_list = []  # Liste pour stocker les images traitées

y_line_coord = int(1500 * scale_percent/100)       # Coordonnée y de la ligne de référence
x_direction_coord = int(2000 * scale_percent/100)  # Coordonnée x du point de référence
offset = int(8 * scale_percent/100)    # Décalage pour la détection des objets par rapport à la ligne
in_counter = 0    # Compteur d'objets entrant
out_counter = 0   # Compteur d'objets sortant

# Informations originales de la vidéo
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = video.get(cv2.CAP_PROP_FPS)
print('[INFO] - Original Dim: ', (width, height))

# Redimensionnement de la vidéo pour de meilleures performances
if scale_percent != 100:
    print('[INFO] - Scaling change may cause errors in pixels lines ')
    width = int(width * scale_percent / 100)
    height = int(height * scale_percent / 100)
    print('[INFO] - Dim Scaled: ', (width, height))

In [ ]:
# Vidéo de sortie
video_name = 'result.mp4'
output_path = "rep_" + video_name
tmp_output_path = "tmp_" + output_path
VIDEO_CODEC = "mp4v" 

output_video = cv2.VideoWriter(tmp_output_path, cv2.VideoWriter_fourcc(*VIDEO_CODEC), fps, (width, height))

# Exécution de la reconnaissance
for i in tqdm(range(int(video.get(cv2.CAP_PROP_FRAME_COUNT)))):
    ret, frame = video.read()

    # Vérification si "frame" est lue avec succès
    if not ret:
        break

    frame = resize_frame(frame, scale_percent)

    if verbose:
        print('Dimension Scaled(frame): ', (frame.shape[1], frame.shape[0]))
        
        
    # Utilisation du modèle YOLO pour prédire les objets dans la trame
    y_hat = model.predict(frame, conf=0.7, classes=class_IDS, device='cpu', verbose=False)
    # Obtention des coordonnées des bounding boxes, confiance, et classes prédites
    boxes = y_hat[0].boxes.xyxy.cpu().numpy()
    conf = y_hat[0].boxes.conf.cpu().numpy()
    classes = y_hat[0].boxes.cls.cpu().numpy() 

    # Affiche la forme des bounding boxes
    print(boxes.shape)

    # Mise à jour des noms des colonnes en fonction de la forme des bboxes
    if boxes.shape[1] == 4:
        columns = ['xmin', 'ymin', 'xmax', 'ymax']
    else:
        raise ValueError("Unexpected shape of bounding boxes data")

    # Création d'un DataFrame avec les coordonnées des bboxes
    positions_frame = pd.DataFrame(boxes, columns=columns)
    labels = [dict_classes[i] for i in classes]

    # Dessine une ligne de comptage sur la frame
    cv2.line(frame, (0, y_line_coord), (int(4500 * scale_percent/100 ), y_line_coord), (255,255,0), 8)

    # Traitement des résultats de la détection pour chaque bboxes
    for ix, row in enumerate(positions_frame.iterrows()):
        xmin, ymin, xmax, ymax = row[1].astype('int')
        center_x, center_y = int(((xmax+xmin))/2), int((ymax+ ymin)/2)

        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (255,0,0), 5)
        cv2.circle(frame, (center_x,center_y), 5,(255,0,0),-1)

        # Affiche les informations sur l'objet (étiquette et confiance)
        cv2.putText(img=frame, text=labels[ix]+' - '+str(np.round(conf[ix],2)),
                org= (xmin,ymin-10), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(255, 0, 0),thickness=2)

        # Catégorie de l'objet pour le comptage
        category = classes[ix] 
        # Vérifie la position de l'objet par rapport à la ligne de comptage
        if (center_y < (y_line_coord + offset)) and (center_y > (y_line_coord - offset)):
            if  (center_x >= 0) and (center_x <= x_direction_coord):
                in_counter += 1
                in_objects_counter[category] += 1
            else:
                out_counter += 1
                out_objects_counter[category] += 1
                
    # Crée des listes pour l'affichage des compteurs           
    in_counter_plt = [f'{dict_classes[k]}: {i}' for k, i in in_objects_counter.items()]
    out_counter_plt = [f'{dict_classes[k]}: {i}' for k, i in out_objects_counter.items()]
    
    # Affiche les compteurs dans la frame vidéo
    cv2.putText(img=frame, text='Vehicles In :', org= (30,30), fontFace=cv2.FONT_HERSHEY_TRIPLEX, 
                fontScale=1, color=(255, 255, 0),thickness=2)

    cv2.putText(img=frame, text='Vehicles Out :', org= (int(2800 * scale_percent/100 ),30), 
                fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(255, 255, 0),thickness=2)

    xt = 40
    for txt in range(len(in_counter_plt)):
        xt +=30
        cv2.putText(img=frame, text=in_counter_plt[txt], org= (30,xt), fontFace=cv2.FONT_HERSHEY_TRIPLEX, 
                    fontScale=1, color=(255, 255, 0),thickness=2)

        cv2.putText(img=frame, text=out_counter_plt[txt], org= (int(2800 * scale_percent/100 ),xt),
                    fontFace=cv2.FONT_HERSHEY_TRIPLEX,fontScale=1, color=(255, 255, 0),thickness=2)

    cv2.putText(img=frame, text=f'In:{in_counter}', org= (int(1820 * scale_percent/100 ),y_line_coord+60),
                fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(255, 255, 0),thickness=2)

    cv2.putText(img=frame, text=f'Out:{out_counter}', org= (int(1800 * scale_percent/100 ),y_line_coord-40),
                fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(255, 255, 0),thickness=2)

    if verbose:
        print(in_counter, out_counter)

    frames_list.append(frame)
    output_video.write(frame)

# Ferme la vidéo   
output_video.release()

# Chemin complet vers l'exécutable ffmpeg
ffmpeg_path = "/opt/homebrew/bin/ffmpeg"

# Supprime le fichier de sortie s'il existe déjà
if os.path.exists(output_path):
    os.remove(output_path)
    
# Utilise subprocess.run pour exécuter la commande ffmpeg avec des options spécifiques
subprocess.run([ffmpeg_path, "-i", tmp_output_path, "-crf", "18", "-preset", "veryfast", "-hide_banner",
    "-loglevel", "error", "-vcodec", "libx264", output_path])

# Supprime le fichier temporaire de sortie
os.remove(tmp_output_path)

# Sampling Transformed Frames Results

In [ ]:
# Checking samples of processed frames
for i in [28, 29, 58]:
    plt.figure(figsize =( 14, 10))
    plt.imshow(frames_list[i])
    plt.show()

# Executing Result Video

In [ ]:
#output video result
frac = 0.7 
Video(data='rep_result.mp4', embed=True, height=int(720 * frac), width=int(1280 * frac))